# 03. Metadata Layer Deep Dive

Iceberg 테이블의 **Metadata Layer**를 구성하는 metadata.json, Manifest List, Manifest File을 직접 읽어보며 그 내부 구조를 분석합니다.

## Metadata Layer 구조

Iceberg의 Metadata Layer는 **3단계 계층 구조**로 이루어져 있습니다.

```
metadata.json (Table Metadata)
  │
  │  current-snapshot-id 로 현재 스냅샷을 가리킴
  │
  ├─▶ snap-{id}.avro (Manifest List)
  │     │
  │     │  각 manifest file의 경로와 요약 통계
  │     │
  │     ├─▶ {hash}.avro (Manifest File)
  │     │     └─ data file 경로, record_count, column_sizes,
  │     │        value_counts, null_value_counts,
  │     │        lower_bounds, upper_bounds
  │     │
  │     └─▶ {hash}.avro (Manifest File)
  │           └─ ...
  │
  └─▶ (이전 스냅샷의 manifest list)
```

### 각 파일의 역할

| 파일 | 포맷 | 내용 |
|------|------|------|
| **metadata.json** | JSON | 테이블의 전체 상태: 스키마, 파티션 스펙, 스냅샷 목록, 현재 스냅샷 ID |
| **snap-*.avro** | Avro | Manifest List: 이 스냅샷에 포함되는 manifest file 목록 + 파티션 요약 통계 |
| **\*.avro** | Avro | Manifest File: 데이터 파일별 상세 정보 (경로, 레코드 수, 컬럼 통계 등) |

## 환경 설정

In [1]:
import sys
sys.path.append('..')

from utils.spark_setup import create_spark_session
from utils.data_generator import generate_orders, to_spark_df
from utils.file_explorer import show_tree

In [2]:
spark = create_spark_session("MetadataDeepDive")

Spark + Iceberg 세션 준비 완료 (warehouse: file:///home/jovyan/data/warehouse)


## 테이블 생성 및 여러 스냅샷 만들기

5번의 INSERT → 1번 UPDATE → 1번 INSERT 순으로 작업하여 **7개의 스냅샷**을 생성합니다.

핵심 포인트: **각 append는 새로운 manifest file을 만들고, 최신 스냅샷의 manifest list는 이전 manifest file까지 모두 포함**합니다.

In [3]:
spark.sql("CREATE DATABASE IF NOT EXISTS demo.lab")
spark.sql("DROP TABLE IF EXISTS demo.lab.meta_orders")

spark.sql("""
    CREATE TABLE demo.lab.meta_orders (
        order_id     BIGINT,
        customer_id  BIGINT,
        product_name STRING,
        order_date   DATE,
        amount       DOUBLE,
        status       STRING
    )
    USING iceberg
    PARTITIONED BY (months(order_date))
""")

print("테이블 생성 완료")

테이블 생성 완료


In [4]:
# Snapshot 1~5: 각 200건씩 5번 INSERT
# 각 append마다 새 manifest file이 생기고, parquet 3개(월별)가 추가됩니다
for i in range(5):
    orders = generate_orders(num_records=200, seed=i, id_offset=i*200+1)
    df = to_spark_df(spark, orders)
    df.writeTo("demo.lab.meta_orders").append()
    print(f"Snapshot {i+1}: 200건 INSERT (id {i*200+1}~{(i+1)*200})")

print(f"\n현재 총 레코드: {spark.sql('SELECT COUNT(*) FROM demo.lab.meta_orders').collect()[0][0]}건")

Snapshot 1: 200건 INSERT (id 1~200)
Snapshot 2: 200건 INSERT (id 201~400)
Snapshot 3: 200건 INSERT (id 401~600)
Snapshot 4: 200건 INSERT (id 601~800)
Snapshot 5: 200건 INSERT (id 801~1000)

현재 총 레코드: 1000건


In [5]:
# Snapshot 6: UPDATE (COW 방식 — 기존 파일을 재작성)
spark.sql("""
    UPDATE demo.lab.meta_orders
    SET status = 'cancelled', amount = 0.0
    WHERE order_id <= 10
""")
print("Snapshot 6: 10건 UPDATE (COW → 기존 manifest에 DELETE 마크 + 새 manifest 생성)")

Snapshot 6: 10건 UPDATE (COW → 기존 manifest에 DELETE 마크 + 새 manifest 생성)


In [6]:
# Snapshot 7: 추가 INSERT
orders_extra = generate_orders(num_records=100, id_offset=1001, seed=99)
df_extra = to_spark_df(spark, orders_extra)
df_extra.writeTo("demo.lab.meta_orders").append()
print("Snapshot 7: 100건 추가 INSERT")

Snapshot 7: 100건 추가 INSERT


In [7]:
# 현재 상태 확인
count = spark.sql("SELECT COUNT(*) AS cnt FROM demo.lab.meta_orders").collect()[0]["cnt"]
print(f"현재 총 레코드: {count}건\n")

print("=== 스냅샷 히스토리 ===")
spark.sql("""
SELECT 
    snapshot_id,
    committed_at,
    operation,
    summary['added-data-files'] as added_files,
    summary['deleted-data-files'] as deleted_files,
    summary['total-records'] as total_records,
    summary['total-data-files'] as total_files
FROM demo.lab.meta_orders.snapshots
ORDER BY committed_at
""").show(truncate=False)

현재 총 레코드: 1100건

=== 스냅샷 히스토리 ===
+-------------------+-----------------------+---------+-----------+-------------+-------------+-----------+
|snapshot_id        |committed_at           |operation|added_files|deleted_files|total_records|total_files|
+-------------------+-----------------------+---------+-----------+-------------+-------------+-----------+
|5152430064774336323|2026-02-09 12:19:30.652|append   |3          |null         |200          |3          |
|399844344297690298 |2026-02-09 12:19:31.321|append   |3          |null         |400          |6          |
|7825674591142968514|2026-02-09 12:19:31.799|append   |3          |null         |600          |9          |
|6798988369621787229|2026-02-09 12:19:32.167|append   |3          |null         |800          |12         |
|2210344528706639574|2026-02-09 12:19:32.517|append   |3          |null         |1000         |15         |
|7989315118847202289|2026-02-09 12:19:33.878|overwrite|3          |3            |1000         |15     

## metadata/ 디렉토리 구조 확인

In [8]:
TABLE_PATH = "/home/jovyan/data/warehouse/lab/meta_orders"
METADATA_PATH = f"{TABLE_PATH}/metadata"

show_tree(METADATA_PATH)

├── 03e80ee8-3437-4379-bd67-f9dcff636446-m0.avro  (7.3 KB)
├── 1f71eb3d-7894-4a79-80ed-51637fb3c8da-m0.avro  (7.3 KB)
├── 46a9f4f5-0fdd-4c30-badb-1b6ee6c87cec-m0.avro  (7.3 KB)
├── 46a9f4f5-0fdd-4c30-badb-1b6ee6c87cec-m1.avro  (7.3 KB)
├── 63efcd48-aa2a-4288-b6f4-92951a812577-m0.avro  (7.3 KB)
├── 78dcda5c-8411-42c8-9621-8e491b0fa56d-m0.avro  (7.3 KB)
├── ad5bdd13-3814-492b-b033-9adc66cc20fb-m0.avro  (7.3 KB)
├── dacd7327-4067-4f67-8bba-306d6f734f6b-m0.avro  (7.3 KB)
├── snap-2210344528706639574-1-63efcd48-aa2a-4288-b6f4-92951a812577.avro  (4.4 KB)
├── snap-399844344297690298-1-03e80ee8-3437-4379-bd67-f9dcff636446.avro  (4.2 KB)
├── snap-5152430064774336323-1-ad5bdd13-3814-492b-b033-9adc66cc20fb.avro  (4.2 KB)
├── snap-5608016557046021933-1-78dcda5c-8411-42c8-9621-8e491b0fa56d.avro  (4.4 KB)
├── snap-6798988369621787229-1-dacd7327-4067-4f67-8bba-306d6f734f6b.avro  (4.3 KB)
├── snap-7825674591142968514-1-1f71eb3d-7894-4a79-80ed-51637fb3c8da.avro  (4.3 KB)
├── snap-7989315118847202289-1-

## 1단계: metadata.json 분석

`metadata.json`은 테이블의 **전체 상태**를 담고 있는 핵심 파일입니다.  
가장 최신 버전의 metadata.json을 직접 읽어봅니다.

In [9]:
import json
import glob
import os

# 가장 최신 metadata.json 찾기
metadata_files = sorted(glob.glob(f"{METADATA_PATH}/v*.metadata.json"))
latest_metadata = metadata_files[-1]
print(f"최신 metadata 파일: {os.path.basename(latest_metadata)}")
print(f"전체 metadata 파일 수: {len(metadata_files)}개 (= 스냅샷 수 + 테이블 생성)")

with open(latest_metadata) as f:
    meta = json.load(f)

최신 metadata 파일: v8.metadata.json
전체 metadata 파일 수: 8개 (= 스냅샷 수 + 테이블 생성)


In [10]:
# 주요 최상위 필드
print("[ metadata.json 최상위 필드 ]\n")
print(f"  format-version:      {meta['format-version']}")
print(f"  table-uuid:          {meta['table-uuid']}")
print(f"  location:            {meta['location']}")
print(f"  current-schema-id:   {meta.get('current-schema-id', 'N/A')}")
print(f"  current-snapshot-id: {meta.get('current-snapshot-id', 'N/A')}")
print(f"  스키마 버전 수:      {len(meta.get('schemas', []))}")
print(f"  스냅샷 수:           {len(meta.get('snapshots', []))}")
print(f"  파티션 스펙 수:      {len(meta.get('partition-specs', []))}")

[ metadata.json 최상위 필드 ]

  format-version:      2
  table-uuid:          c60f5c0c-0419-41cc-bc1b-a33e0ea10333
  location:            file:///home/jovyan/data/warehouse/lab/meta_orders
  current-schema-id:   0
  current-snapshot-id: 5608016557046021933
  스키마 버전 수:      1
  스냅샷 수:           7
  파티션 스펙 수:      1


In [11]:
# 스키마 정보
print("[ schemas ]\n")
for schema in meta.get('schemas', []):
    print(f"  Schema ID: {schema['schema-id']}")
    for field in schema['fields']:
        print(f"    - id={field['id']}, name={field['name']}, type={field['type']}, required={field['required']}")
    print()

[ schemas ]

  Schema ID: 0
    - id=1, name=order_id, type=long, required=False
    - id=2, name=customer_id, type=long, required=False
    - id=3, name=product_name, type=string, required=False
    - id=4, name=order_date, type=date, required=False
    - id=5, name=amount, type=double, required=False
    - id=6, name=status, type=string, required=False



In [12]:
# 파티션 스펙
print("[ partition-specs ]\n")
for spec in meta.get('partition-specs', []):
    print(f"  Spec ID: {spec['spec-id']}")
    for field in spec['fields']:
        print(f"    - source-id={field['source-id']}, name={field['name']}, transform={field['transform']}")

[ partition-specs ]

  Spec ID: 0
    - source-id=4, name=order_date_month, transform=month


In [13]:
# 스냅샷 목록 — summary 키가 다를 수 있으므로 유연하게 처리
print("[ snapshots ]\n")
for snap in meta.get('snapshots', []):
    summary = snap.get('summary', {})
    print(f"  Snapshot ID:   {snap['snapshot-id']}")
    print(f"  Timestamp:     {snap['timestamp-ms']}")
    print(f"  Operation:     {summary.get('operation', 'N/A')}")
    print(f"  Manifest List: {os.path.basename(snap['manifest-list'])}")
    print(f"  Summary:       added-files={summary.get('added-data-files', '?')}, "
          f"deleted-files={summary.get('deleted-data-files', '0')}, "
          f"total-records={summary.get('total-records', '?')}")
    print()

[ snapshots ]

  Snapshot ID:   5152430064774336323
  Timestamp:     1770639570652
  Operation:     append
  Manifest List: snap-5152430064774336323-1-ad5bdd13-3814-492b-b033-9adc66cc20fb.avro
  Summary:       added-files=3, deleted-files=0, total-records=200

  Snapshot ID:   399844344297690298
  Timestamp:     1770639571321
  Operation:     append
  Manifest List: snap-399844344297690298-1-03e80ee8-3437-4379-bd67-f9dcff636446.avro
  Summary:       added-files=3, deleted-files=0, total-records=400

  Snapshot ID:   7825674591142968514
  Timestamp:     1770639571799
  Operation:     append
  Manifest List: snap-7825674591142968514-1-1f71eb3d-7894-4a79-80ed-51637fb3c8da.avro
  Summary:       added-files=3, deleted-files=0, total-records=600

  Snapshot ID:   6798988369621787229
  Timestamp:     1770639572167
  Operation:     append
  Manifest List: snap-6798988369621787229-1-dacd7327-4067-4f67-8bba-306d6f734f6b.avro
  Summary:       added-files=3, deleted-files=0, total-records=800

  S

## 2단계: Manifest List (snap-*.avro) 분석

Manifest List는 스냅샷에 포함되는 **manifest file들의 목록**을 담고 있습니다.

핵심: **최신 스냅샷의 manifest list에는 이전 append들의 manifest file도 모두 포함**됩니다.

```
Snapshot 7의 Manifest List (snap-xxx.avro)
  ├── manifest-7.avro  ← Snapshot 7에서 추가된 parquet 3개
  ├── manifest-6b.avro ← UPDATE로 새로 쓴 parquet (added)
  ├── manifest-6a.avro ← UPDATE로 삭제 마크된 기존 parquet (deleted)
  ├── manifest-5.avro  ← Snapshot 5에서 추가된 parquet 3개
  ├── manifest-4.avro  ← Snapshot 4에서 추가된 parquet 3개
  ├── manifest-3.avro  ← ...
  ├── manifest-2.avro  
  └── manifest-1.avro  ← Snapshot 1에서 추가된 parquet 3개
```

`fastavro`를 사용하여 직접 읽어봅니다.

In [14]:
import fastavro

def strip_file_uri(path):
    """URI scheme 제거: 'file:/path' 또는 'file:///path' → '/path'"""
    if path.startswith('file:'):
        return path.replace('file:///', '/').replace('file:/', '/')
    return path

# 현재 스냅샷의 manifest list 찾기
current_snap = None
current_snap_id = meta.get('current-snapshot-id')
for snap in meta.get('snapshots', []):
    if snap['snapshot-id'] == current_snap_id:
        current_snap = snap
        break

manifest_list_path = strip_file_uri(current_snap['manifest-list'])
print(f"현재 스냅샷(ID: {current_snap_id})의 Manifest List:")
print(f"  {os.path.basename(manifest_list_path)}\n")

with open(manifest_list_path, 'rb') as f:
    reader = fastavro.reader(f)
    manifest_list_records = list(reader)

print(f"이 Manifest List가 참조하는 Manifest File 수: {len(manifest_list_records)}개")
print("=" * 70)

for i, record in enumerate(manifest_list_records):
    added = record.get('added_data_files_count', 0)
    existing = record.get('existing_data_files_count', 0)
    deleted = record.get('deleted_data_files_count', 0)
    
    # 역할 판단
    if added > 0 and deleted == 0:
        role = "📥 데이터 추가 (append)"
    elif added > 0 and deleted > 0:
        role = "🔄 데이터 재작성 (overwrite - added)"
    elif deleted > 0 and added == 0:
        role = "🗑️ 삭제 마크 (overwrite - deleted)"
    else:
        role = f"📄 existing={existing}"
    
    print(f"\n  [{i}] {os.path.basename(record['manifest_path'])}")
    print(f"      added={added}, existing={existing}, deleted={deleted}")
    print(f"      → {role}")

현재 스냅샷(ID: 5608016557046021933)의 Manifest List:
  snap-5608016557046021933-1-78dcda5c-8411-42c8-9621-8e491b0fa56d.avro

이 Manifest List가 참조하는 Manifest File 수: 6개

  [0] 78dcda5c-8411-42c8-9621-8e491b0fa56d-m0.avro
      added=3, existing=0, deleted=0
      → 📥 데이터 추가 (append)

  [1] 46a9f4f5-0fdd-4c30-badb-1b6ee6c87cec-m1.avro
      added=3, existing=0, deleted=0
      → 📥 데이터 추가 (append)

  [2] 63efcd48-aa2a-4288-b6f4-92951a812577-m0.avro
      added=3, existing=0, deleted=0
      → 📥 데이터 추가 (append)

  [3] dacd7327-4067-4f67-8bba-306d6f734f6b-m0.avro
      added=3, existing=0, deleted=0
      → 📥 데이터 추가 (append)

  [4] 1f71eb3d-7894-4a79-80ed-51637fb3c8da-m0.avro
      added=3, existing=0, deleted=0
      → 📥 데이터 추가 (append)

  [5] 03e80ee8-3437-4379-bd67-f9dcff636446-m0.avro
      added=3, existing=0, deleted=0
      → 📥 데이터 추가 (append)


## 3단계: Manifest File (*.avro) 분석

Manifest File은 개별 **데이터 파일의 상세 정보**를 담고 있습니다.
여기에 **컬럼 수준 통계**(min/max, null count 등)가 저장됩니다.

이제 manifest list가 참조하는 **각 manifest file이 어떤 parquet 파일들을 가리키는지** 전체 계층을 펼쳐봅니다.

In [15]:
# 전체 계층 시각화: Manifest List → Manifest Files → Parquet Files
print(f"Manifest List: {os.path.basename(manifest_list_path)}")
print(f"  (Manifest File {len(manifest_list_records)}개를 참조)")
print("=" * 80)

all_manifest_records = {}  # 나중에 상세 분석용

for i, ml_record in enumerate(manifest_list_records):
    manifest_path = strip_file_uri(ml_record['manifest_path'])
    manifest_name = os.path.basename(manifest_path)
    added = ml_record.get('added_data_files_count', 0)
    existing = ml_record.get('existing_data_files_count', 0)
    deleted = ml_record.get('deleted_data_files_count', 0)
    
    connector = "└── " if i == len(manifest_list_records) - 1 else "├── "
    prefix = "    " if i == len(manifest_list_records) - 1 else "│   "
    
    print(f"\n{connector}[Manifest {i}] {manifest_name}")
    print(f"{prefix}  added={added}, existing={existing}, deleted={deleted}")
    
    # 각 manifest file 열어서 parquet 파일 목록 출력
    with open(manifest_path, 'rb') as f:
        reader = fastavro.reader(f)
        entries = list(reader)
    
    all_manifest_records[i] = entries
    
    for j, entry in enumerate(entries):
        df_info = entry.get('data_file', entry)
        if isinstance(df_info, dict):
            file_path = os.path.basename(str(df_info.get('file_path', '?')))
            record_count = df_info.get('record_count', '?')
            file_size = df_info.get('file_size_in_bytes', '?')
            status = entry.get('status', '?')  # 0=EXISTING, 1=ADDED, 2=DELETED
            status_label = {0: "EXISTING", 1: "ADDED", 2: "DELETED"}.get(status, str(status))
            
            sub_connector = "└── " if j == len(entries) - 1 else "├── "
            print(f"{prefix}  {sub_connector}{file_path}  ({record_count}행, {file_size}B) [{status_label}]")

Manifest List: snap-5608016557046021933-1-78dcda5c-8411-42c8-9621-8e491b0fa56d.avro
  (Manifest File 6개를 참조)

├── [Manifest 0] 78dcda5c-8411-42c8-9621-8e491b0fa56d-m0.avro
│     added=3, existing=0, deleted=0
│     ├── 00000-40-c38747bb-637d-4173-8de3-cbbec3d0b85f-00001.parquet  (38행, 2614B) [ADDED]
│     ├── 00000-40-c38747bb-637d-4173-8de3-cbbec3d0b85f-00002.parquet  (26행, 2495B) [ADDED]
│     └── 00000-40-c38747bb-637d-4173-8de3-cbbec3d0b85f-00003.parquet  (36행, 2608B) [ADDED]

├── [Manifest 1] 46a9f4f5-0fdd-4c30-badb-1b6ee6c87cec-m1.avro
│     added=3, existing=0, deleted=0
│     ├── 00000-34-7982c214-f52a-44b2-914e-7ab0532ec29f-00001.parquet  (69행, 2928B) [ADDED]
│     ├── 00000-34-7982c214-f52a-44b2-914e-7ab0532ec29f-00002.parquet  (65행, 2934B) [ADDED]
│     └── 00000-34-7982c214-f52a-44b2-914e-7ab0532ec29f-00003.parquet  (66행, 2898B) [ADDED]

├── [Manifest 2] 63efcd48-aa2a-4288-b6f4-92951a812577-m0.avro
│     added=3, existing=0, deleted=0
│     ├── 00000-29-bfd8f3eb-3134-4f9b-a

In [16]:
import struct
from datetime import date, timedelta

# Iceberg 바이너리 → 사람이 읽을 수 있는 값으로 변환
DECODERS = {
    'long':    lambda b: struct.unpack('<q', b)[0],
    'int':     lambda b: struct.unpack('<i', b)[0],
    'float':   lambda b: struct.unpack('<f', b)[0],
    'double':  lambda b: struct.unpack('<d', b)[0],
    'string':  lambda b: b.decode('utf-8'),
    'boolean': lambda b: bool(b[0]),
    'date':    lambda b: date(1970, 1, 1) + timedelta(days=struct.unpack('<i', b)[0]),
}

# metadata.json의 스키마에서 컬럼 ID → (이름, 타입) 매핑 생성
schema_fields = meta['schemas'][0]['fields']
col_info = {f['id']: (f['name'], f['type']) for f in schema_fields}

def decode_bound(col_id, raw_bytes):
    """컬럼 ID와 바이너리 값을 받아 사람이 읽을 수 있는 값으로 변환"""
    name, col_type = col_info.get(col_id, (f'col_{col_id}', 'unknown'))
    decoder = DECODERS.get(col_type)
    if decoder:
        return name, decoder(raw_bytes)
    return name, raw_bytes

# 첫 번째 manifest의 첫 번째 데이터 파일 상세 분석
print("[ 데이터 파일 엔트리 상세 분석 ]\n")

sample_entries = all_manifest_records[0]
entry = sample_entries[0]
df_info = entry.get('data_file', entry)

print(f"파일: {os.path.basename(str(df_info.get('file_path', '?')))}")
print(f"레코드 수: {df_info.get('record_count', '?')}")
print(f"파일 크기: {df_info.get('file_size_in_bytes', '?')} bytes")

# column_sizes
col_sizes = df_info.get('column_sizes', [])
if col_sizes:
    print(f"\ncolumn_sizes:")
    for item in col_sizes:
        name, _ = col_info.get(item['key'], (f"col_{item['key']}", '?'))
        print(f"  {name:20s} {item['value']:>6} bytes")

# value_counts / null_value_counts
val_counts = df_info.get('value_counts', [])
null_counts = df_info.get('null_value_counts', [])
if val_counts:
    null_map = {item['key']: item['value'] for item in null_counts} if null_counts else {}
    print(f"\nvalue_counts / null_counts:")
    for item in val_counts:
        name, _ = col_info.get(item['key'], (f"col_{item['key']}", '?'))
        nulls = null_map.get(item['key'], 0)
        print(f"  {name:20s} values={item['value']}, nulls={nulls}")

# lower_bounds / upper_bounds — 핵심: 바이너리를 사람이 읽을 수 있게 디코딩
lower = df_info.get('lower_bounds', [])
upper = df_info.get('upper_bounds', [])
if lower and upper:
    upper_map = {item['key']: item['value'] for item in upper}
    print(f"\n{'컬럼':20s} {'min':>20s}    {'max':>20s}")
    print("-" * 65)
    for item in lower:
        col_id = item['key']
        name, lower_val = decode_bound(col_id, item['value'])
        _, upper_val = decode_bound(col_id, upper_map[col_id])
        print(f"  {name:18s} {str(lower_val):>20s}    {str(upper_val):>20s}")

[ 데이터 파일 엔트리 상세 분석 ]

파일: 00000-40-c38747bb-637d-4173-8de3-cbbec3d0b85f-00001.parquet
레코드 수: 38
파일 크기: 2614 bytes

column_sizes:
  order_id                139 bytes
  customer_id             167 bytes
  product_name            223 bytes
  order_date              143 bytes
  amount                  282 bytes
  status                  146 bytes

value_counts / null_counts:
  order_id             values=38, nulls=0
  customer_id          values=38, nulls=0
  product_name         values=38, nulls=0
  order_date           values=38, nulls=0
  amount               values=38, nulls=0
  status               values=38, nulls=0

컬럼                                    min                     max
-----------------------------------------------------------------
  order_id                           1002                    1097
  customer_id                         133                     997
  product_name                    AirPods               iPhone SE
  order_date                   2024-01-01  

## Column-level Statistics와 쿼리 최적화

Manifest File에 저장된 `lower_bounds`와 `upper_bounds`는 **파일 프루닝(File Pruning)**에 사용됩니다.

예를 들어, 다음 쿼리를 실행할 때:

```sql
SELECT * FROM orders WHERE amount > 5000
```

Iceberg는 각 데이터 파일의 `amount` 컬럼 `upper_bounds`를 확인하여:
- `upper_bounds(amount) < 5000`인 파일은 **읽지 않고 건너뜁니다**
- 이를 통해 불필요한 I/O를 줄이고 쿼리 성능을 향상시킵니다

```
파일 A: amount lower=100, upper=3000  → ❌ SKIP (max < 5000)
파일 B: amount lower=200, upper=6000  → ✅ READ (max >= 5000)
파일 C: amount lower=5500, upper=8000 → ✅ READ (max >= 5000)
```

## Snapshot 체인 시각화

여러 작업을 거치면서 스냅샷이 어떻게 쌓이는지 확인합니다.

In [17]:
from datetime import datetime as dt

snapshots = meta.get('snapshots', [])

print("Snapshot 체인:")
print("=" * 70)

for i, snap in enumerate(snapshots):
    snap_id = snap['snapshot-id']
    ts = dt.fromtimestamp(snap['timestamp-ms'] / 1000).strftime('%Y-%m-%d %H:%M:%S')
    summary = snap.get('summary', {})
    op = summary.get('operation', '?')
    total_records = summary.get('total-records', '?')
    total_files = summary.get('total-data-files', '?')
    is_current = " << CURRENT" if snap_id == current_snap_id else ""
    
    if i > 0:
        print("     |")
        print("     v")
    
    print(f"  +--- Snapshot v{i+1}{is_current}")
    print(f"  | ID:        {snap_id}")
    print(f"  | Time:      {ts}")
    print(f"  | Operation: {op}")
    print(f"  | Records:   {total_records} ({total_files} files)")
    print(f"  +---")

Snapshot 체인:
  +--- Snapshot v1
  | ID:        5152430064774336323
  | Time:      2026-02-09 12:19:30
  | Operation: append
  | Records:   200 (3 files)
  +---
     |
     v
  +--- Snapshot v2
  | ID:        399844344297690298
  | Time:      2026-02-09 12:19:31
  | Operation: append
  | Records:   400 (6 files)
  +---
     |
     v
  +--- Snapshot v3
  | ID:        7825674591142968514
  | Time:      2026-02-09 12:19:31
  | Operation: append
  | Records:   600 (9 files)
  +---
     |
     v
  +--- Snapshot v4
  | ID:        6798988369621787229
  | Time:      2026-02-09 12:19:32
  | Operation: append
  | Records:   800 (12 files)
  +---
     |
     v
  +--- Snapshot v5
  | ID:        2210344528706639574
  | Time:      2026-02-09 12:19:32
  | Operation: append
  | Records:   1000 (15 files)
  +---
     |
     v
  +--- Snapshot v6
  | ID:        7989315118847202289
  | Time:      2026-02-09 12:19:33
  | Operation: overwrite
  | Records:   1000 (15 files)
  +---
     |
     v
  +--- Snapsho

## 관찰 포인트

### Metadata Layer의 3단계 계층

```
metadata.json
  └─▶ snap-xxx.avro (Manifest List)
        ├─▶ manifest-7.avro  → [parquet-7a, 7b, 7c]  ← Snapshot 7 (append)
        ├─▶ manifest-6b.avro → [parquet-6x, 6y, 6z]  ← Snapshot 6 (UPDATE - 새 파일)
        ├─▶ manifest-6a.avro → [parquet-1a DELETED]   ← Snapshot 6 (UPDATE - 삭제 마크)
        ├─▶ manifest-5.avro  → [parquet-5a, 5b, 5c]  ← Snapshot 5
        ├─▶ manifest-4.avro  → [parquet-4a, 4b, 4c]  ← Snapshot 4
        ├─▶ manifest-3.avro  → [parquet-3a, 3b, 3c]  ← Snapshot 3
        ├─▶ manifest-2.avro  → [parquet-2a, 2b, 2c]  ← Snapshot 2
        └─▶ manifest-1.avro  → [parquet-1a, 1b, 1c]  ← Snapshot 1
```

### 핵심 인사이트

1. **metadata.json**: 모든 스냅샷의 "목차". 현재 스냅샷 ID를 통해 최신 manifest list를 찾음
2. **Manifest List**: 현재 스냅샷의 **전체 데이터**를 구성하는 manifest file 목록. 이전 배치의 manifest도 포함
3. **Manifest File**: 각 배치에서 추가/삭제된 parquet 파일 목록 + 컬럼 통계

### UPDATE가 만드는 구조

- COW(Copy-on-Write) UPDATE는 **기존 파일을 DELETED로 마크**하고 **새 파일을 ADDED로 기록**
- 삭제 마크된 파일은 manifest에 `status=DELETED`로 남아있다가, `expire_snapshots` 후 물리적으로 삭제됨

### 쿼리 최적화에 미치는 영향

- Manifest List의 파티션 요약 → **파티션 프루닝** (파티션 통째로 스킵)
- Manifest File의 min/max 통계 → **파일 프루닝** (파일 단위로 스킵)
- 이 두 단계를 거쳐 **실제로 읽어야 할 parquet 파일 수를 최소화**

In [18]:
spark.stop()
print("Spark 세션 종료")

Spark 세션 종료
